In [ ]:
# Pandas dibutuhkan untuk manipulasi dan transformasi data
import pandas as pd

import numpy as np
# Sklearn digunakan untuk segala hal yang berhubungan dengan machine learning modeling
# train_test_split : untuk membagi data train menjadi data untuk training dan validasi
from sklearn.model_selection import train_test_split

# DecisionTreeClassifier : contoh model yang digunakan disini yaitu Decision Tree
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import confusion_matrix

from sklearn.preprocessing import OneHotEncoder

from sklearn.ensemble import RandomForestClassifier

from sklearn.svm import SVC

from sklearn.preprocessing import LabelEncoder

In [ ]:
from datetime import datetime

def days_between(d1, d2):
    d1 = datetime.strptime(d1, "%Y-%m-%d")
    d2 = datetime.strptime(d2, "%Y-%m-%d")
    return abs((d2 - d1).days)


In [ ]:
# Data Preparation
df = pd.read_csv('sampling_healtkathon2022/sampling_healtkathon2022.csv')

In [ ]:
rand_processed = df

In [ ]:
# Bikin labelencoder
labelencoder = LabelEncoder()

# Ngelabel encoding
rand_processed['jenkel_cat'] = labelencoder.fit_transform(rand_processed['jenkel'])

rand_processed['politujuan_cat'] = labelencoder.fit_transform(rand_processed['politujuan'])
rand_processed['typefaskes_cat'] = labelencoder.fit_transform(rand_processed['typefaskes'])
rand_processed['cbg_cat'] = labelencoder.fit_transform(rand_processed['cbg'])
rand_processed['diagfktp_cat'] = labelencoder.fit_transform(rand_processed['diagfktp'])

rand_processed['kdsa_cat'] = labelencoder.fit_transform(rand_processed['kdsa'])
rand_processed['kdsp_cat'] = labelencoder.fit_transform(rand_processed['kdsp'])
rand_processed['kdsr_cat'] = labelencoder.fit_transform(rand_processed['kdsr'])
rand_processed['kdsi_cat'] = labelencoder.fit_transform(rand_processed['kdsi'])
rand_processed['kdsd_cat'] = labelencoder.fit_transform(rand_processed['kdsd'])

# Drop semua yang gak dibutuhin
rand_processed = rand_processed.drop(columns=['id'])
rand_processed = rand_processed.drop(columns=['id_peserta'])

rand_processed = rand_processed.drop(columns=['jenkel'])

rand_processed = rand_processed.drop(columns=['politujuan'])
rand_processed = rand_processed.drop(columns=['typefaskes'])
rand_processed = rand_processed.drop(columns=['cbg'])
rand_processed = rand_processed.drop(columns=['diagfktp'])

rand_processed = rand_processed.drop(columns=['kdsa'])
rand_processed = rand_processed.drop(columns=['kdsp'])
rand_processed = rand_processed.drop(columns=['kdsr'])
rand_processed = rand_processed.drop(columns=['kdsi'])
rand_processed = rand_processed.drop(columns=['kdsd'])

# Drop semua yang gak dibutuhin
rand_processed['biaya_bagi100'] = rand_processed['biaya']/100
rand_processed = rand_processed.drop(columns=['biaya'])

rand_processed

In [ ]:
# Isi semua na
no_na = rand_processed
no_na['biaya_bagi100'] = no_na['biaya_bagi100'].fillna(no_na['biaya_bagi100'].mean())
no_na = no_na.fillna(0)

no_na

In [ ]:
no_na["Selisih"] = " "

for i in range(len(no_na)) :
     
    x = no_na.iloc[i]['tgldatang']
    y = no_na.iloc[i]['tglpulang']

    if x == y :
        no_na.at[i,'Selisih'] = 0

    else :

        TanggalX = x[0:10]
        TanggalY = y[0:10]

        Selisih = days_between(TanggalX, TanggalY)
        no_na.at[i, 'Selisih'] = Selisih



In [ ]:
no_na.describe()
#rand_processed.isna().sum()

In [ ]:
Pred = no_na.drop(columns=['label'])


In [ ]:
Pred = Pred.select_dtypes(exclude=['object'])

In [ ]:
# Pembagian Batch

Batch1, Batch2, Batch3, Batch4, Batch5 = np.array_split(Pred, 5)

In [ ]:
# Pembuatan data training

satu = no_na.loc[no_na['label'] == 1] # Pengambilan yang labelnya 1
nolfull = no_na.loc[no_na['label'] == 0] # Pengambilan yang label 0

nol = nolfull.sample(n=len(satu)) # Pengambilan label 0 sejumlah banyaknya label 1



In [ ]:
Train = nol.append(satu)
Train = Train.sample(frac = 1)
Train = Train.select_dtypes(exclude=['object'])

In [ ]:
X = Train.drop(columns=['label'])
y = Train.label

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# svc = SVC(probability = True)
# svc = svc.fit(X_train, y_train)

In [ ]:
# Hasilpred = svc.predict(Pred)

In [ ]:
# rf = RandomForestClassifier(random_state = 1)
# rf = rf.fit(X_train, y_train)

In [ ]:
# Hasilpred = rf.predict(Pred)

In [ ]:
clf = DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

In [ ]:
Hasilpred1 = clf.predict(Batch1)

In [ ]:
Hasilpred2 = clf.predict(Batch2)

In [ ]:
Hasilpred3 = clf.predict(Batch3)

In [ ]:
Hasilpred4 = clf.predict(Batch4)

In [ ]:
Hasilpred5 = clf.predict(Batch5)

In [ ]:
# Evaluation

y_true = rand_processed.label
y_pred = Hasilpred1 + Hasilpred2 + Hasilpred3 + Hasilpred4 + Hasilpred5


def check(true,pred)  :
    tn, fp, fn, tp = confusion_matrix(true, pred).ravel()
    Accuracy = (tn+tp) / (tn+fp+tp+fn)
    Precision = tp/(tp+fp)
    Recall = tp/(tp+fn)
    Specifity = tn/(tn+fp)  

    print("Accuracy    :", Accuracy, "\nPrecision   :", Precision, "\nRecall      :", Recall, "\nSpecifity   :", Specifity)


check(y_true,y_pred)

